# Main Script

In [1]:
# This script does the following
# 1. Downloads OSM files for specified countries from Geofabrik
# 2. Filters files for substations and lines
# 3. Process and clean data
# 4. Exports to CSV
# 5. Exports to GeoJson

"""
OSM extraction scrpt
"""

import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import  osm_colors          as CC
from esy.osmfilter import run_filter, Node,Way,Relation 
from esy.osmfilter import export_geojson
from esy.osmfilter import osm_info             as osm_info
from esy.osmfilter import osm_pickle           as osm_pickle
from contextlib import contextmanager

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt

import logging
logger = logging.getLogger(__name__)



# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/


# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)

# Downloads PBF File for given Country Code

def download_pbf(country_code, update):  # update = true forces re-download of files
    """
    TEST ONLY. FROM PYPSA.
    Initializes variables for power dispatch for a given component and a
    given attribute.

    Parameters
    ----------
    n : pypsa.Network
    c : str
        name of the network component
    attr : str
        name of the attribute, e.g. 'p'

    """
    country_name = AFRICA_CC[country_code]
    # Filename for geofabrik
    geofabrik_filename = f'{country_name}-latest.osm.pbf'
    # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf
    geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}'
    PBF_inputfile = os.path.join(
        os.getcwd(), "data", "osm", "pbf", geofabrik_filename)  # Input filepath

    if not os.path.exists(PBF_inputfile) or update is True:
        print(f"{geofabrik_filename} does not exist, downloading to {PBF_inputfile}")
        # create data/osm directory
        os.makedirs(os.path.dirname(PBF_inputfile), exist_ok=True)
        with requests.get(geofabrik_url, stream=True) as r:
            with open(PBF_inputfile, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

    return PBF_inputfile


def download_and_filter(country_code, update=False):
    PBF_inputfile = download_pbf(country_code, update)

    filter_file_exists = False
    # json file for the Data dictionary
    JSON_outputfile = os.path.join(
        os.getcwd(), 'data', 'osm', country_code+'_power.json')
    # json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)

    if os.path.exists(JSON_outputfile):
        filter_file_exists = True

    # Load Previously Pre-Filtered Files
    if update is False and filter_file_exists is True:
        create_elements = False  # Do not create elements again
        new_prefilter_data = False  # Do not pre-filter data again
        # HACKY: esy.osmfilter code to re-create Data.pickle
        Data = osm_info.ReadJason(JSON_outputfile, verbose='no')
        DataDict = {"Data": Data}
        osm_pickle.picklesave(DataDict, os.path.realpath(
            os.path.join(os.getcwd(), os.path.dirname(JSON_outputfile))))
        print(f'Loading Pickle for {AFRICA_CC[country_code]}')  # TODO: Change to Logger
    else:
        create_elements = True
        new_prefilter_data = True
        print(f'Creating  New Elements for {AFRICA_CC[country_code]}')  # TODO: Change to Logger

    prefilter = {Node: {"power": ["substation", "line","generator"]}, Way: {
        "power": ["substation", "line","generator"]}, Relation: {"power": ["substation", "line","generator"]}} #see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures
    # HACKY: due to esy.osmfilter validation
    blackfilter = [("", ""), ]

    for feature in ["substation", "line","generator"]:
        whitefilter = [[("power", feature), ], ]
        elementname = f'{country_code}_{feature}s'

        feature_data = run_filter(elementname, PBF_inputfile, JSON_outputfile, prefilter, whitefilter, blackfilter,
                                  NewPreFilterData=new_prefilter_data, CreateElements=create_elements, LoadElements=True, verbose=False, multiprocess=True)

        if feature == 'substation':
            substation_data = feature_data
        if feature == 'line':
            line_data = feature_data
        if feature == 'generator':
            generator_data = feature_data

    return (substation_data, line_data, generator_data)

# Convert Ways to Point Coordinates


# TODO: Use shapely and merge with convert_ways_lines
def convert_ways_nodes(df_way, Data):
    lonlat_column = []
    # col = "refs"
    # df_way[col] = pd.Series().astype(float) if col not in df_way.columns else df_way[col] #create empty "refs" if not in dataframe
    for ref in df_way["refs"]:
        lonlats = []
        for r in ref:
            lonlat = Data["Node"][str(r)]["lonlat"]
            lonlat = tuple(lonlat)
            lonlats.append(lonlat)
        lonlats = np.array(lonlats)
        lonlat = np.mean(lonlats, axis=0)  # Hacky Apporx Centroid
        lonlat_column.append(lonlat)
    # df_way.drop('refs', axis=1, inplace=True, errors='ignore')
    df_way.insert(0, "lonlat", lonlat_column)

# Convert Ways to Line Coordinates


def convert_ways_lines(df_way, Data):
    lonlat_column = []
    for ref in df_way["refs"]:  # goes through each row in df_way['refs']
        lonlats = []
        # picks each element in ref & replaces ID by coordinate tuple (A multiline consist of several points)
        for r in ref:
            # "r" is the ID in Data["Node"], ["lonlat"] a list of [x1,y1] (coordinates)
            lonlat = Data["Node"][str(r)]["lonlat"]
            lonlat = tuple(lonlat)
            lonlats.append(lonlat)  # a list with tuples
        lonlat_column.append(lonlats)  # adding a new list of tuples every row
    # df_way.drop('refs', axis=1, inplace=True)
    df_way.insert(1, "lonlat", lonlat_column)

# Convert Points Pandas Dataframe to GeoPandas Dataframe


def convert_pd_to_gdf(df_way):
    gdf = gpd.GeoDataFrame(
        df_way, geometry=[Point(x, y) for x, y in df_way.lonlat])
    gdf.drop(columns=['lonlat'], inplace=True)
    return gdf

# Convert Lines Pandas Dataframe to GeoPandas Dataframe


def convert_pd_to_gdf_lines(df_way, simplified=False):
    df_way['geometry'] = df_way['lonlat'].apply(lambda x: LineString(x))
    if simplified is True:
        df_way['geometry'] = df_way['geometry'].apply(lambda x: x.simplify(0.005, preserve_topology=False))
    gdf = gpd.GeoDataFrame(df_way, geometry="geometry", crs="EPSG:4326")
    gdf.drop(columns=['lonlat'], inplace=True)

    return gdf

# Convert Filtered Data, Elements to Pandas Dataframes


def convert_filtered_data_to_dfs(country_code, feature_data, feature):
    [Data, Elements] = feature_data
    elementname = f'{country_code}_{feature}s'
    df_way = pd.json_normalize(Elements[elementname]["Way"].values())
    df_node = pd.json_normalize(Elements[elementname]["Node"].values())
    return (df_node, df_way, Data)


def process_substation_data(country_code, substation_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(country_code, substation_data, 'substation')
    convert_ways_nodes(df_way, Data)
    # Add Type Column
    df_node['Type'] = 'Node'
    df_way['Type'] = 'Way'

    df_combined = pd.concat([df_node, df_way], axis=0)
    # Add Country Column
    df_combined['Country'] = AFRICA_CC[country_code]

    return df_combined


def process_line_data(country_code, line_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(country_code, line_data, 'line')
    convert_ways_lines(df_way, Data)
    # Add Type Column
    df_way['Type'] = 'Way'

    # Add Country Column
    df_way['Country'] = AFRICA_CC[country_code]
    return df_way


def process_generator_data(country_code, generator_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(country_code, generator_data, 'generator')
    convert_ways_nodes(df_way, Data)
    # Add Type Column
    df_node['Type'] = 'Node'
    df_way['Type'] = 'Way'

    df_combined = pd.concat([df_node, df_way], axis=0)
    # Add Country Column
    df_combined['Country'] = AFRICA_CC[country_code]

    return df_combined


def process_data():
    df_all_substations = pd.DataFrame()
    df_all_lines = pd.DataFrame()
    df_all_generators = pd.DataFrame()
    # test_CC = {"NG": "nigeria"}
    for country_code in AFRICA_CC.keys():
        substation_data, line_data, generator_data = download_and_filter(country_code)
        for feature in ["substation", "line","generator"]:
            if feature == 'substation':
                df_substation = process_substation_data(country_code, substation_data)
                df_all_substations = pd.concat(
                    [df_all_substations, df_substation])
            if feature == 'line':
                df_line = process_line_data(country_code, line_data)
                df_all_lines = pd.concat([df_all_lines, df_line])
            if feature == 'generator':
                df_generator = process_generator_data(country_code, generator_data)
                df_all_generators = pd.concat(
                    [df_all_generators, df_generator])
    
    #----------- SUBSTATIONS -----------

    # Clean
    df_all_substations.reset_index(drop=True, inplace=True)
    df_all_substations.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
    df_all_substations.dropna(thresh=len(df_all_substations)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A

    # Generate Files
    outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations.')
    df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV
    gdf_substations = convert_pd_to_gdf(df_all_substations)
    gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


    # ----------- LINES -----------

    # Clean
    # TODO: FIX Voltage Filter
    # Some transmission lines carry multiple voltages, having voltage_V = 10000;20000  (two lines)
    # The following code keeps only the first information before the semicolon..
    # Needs to be corrected in future, creating two lines with the same bus ID.
    
    df_all_lines.reset_index(drop=True, inplace=True)
    df_all_lines.dropna(subset=['tags.voltage'], inplace = True) # Drop any lines with Voltage = N/A
    df_all_lines.rename(columns = {'tags.voltage':"voltage_V"}, inplace = True) 
    df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split('*').str[0]
    df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split(';').str[0]
    df_all_lines['voltage_V'] = df_all_lines['voltage_V'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna() ## if cell can't converted to float -> drop
    df_all_lines = df_all_lines[df_all_lines.voltage_V > 10000]
    df_all_lines.dropna(thresh=len(df_all_lines)*0.25, axis=1, how='all', inplace=True) # Drop Columns with 75% values as N/A

    # Generate Files
    outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines.')  
    df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV
    gdf_lines = convert_pd_to_gdf_lines(df_all_lines, simplified=True)
    gdf_lines.to_file(outputfile_partial+'geojson',
                driver="GeoJSON")  # Generate GeoJson


    # ----------- Generator -----------
    
    df_all_generators.reset_index(drop=True, inplace=True)
    df_all_generators.drop(columns = ["tags.fixme","tags.frequency","tags.name:ar","tags.building","tags.barrier"], inplace = True, errors='ignore')
    df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
    df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
    df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"}, inplace = True)
    df_all_generators.dropna(thresh=len(df_all_generators)*0.25, axis=1, how='all', inplace=True) # Drop Columns with 75% values as N/A

    # Generate Files
    outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators.')
    df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV
    gdf_generators = convert_pd_to_gdf(df_all_generators)
    gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson




# Overwrite Functions

In [2]:
def convert_ways_lines(df_way, Data):
    lonlat_column = []
    
    for ref in df_way["refs"]:  # goes through each row in df_way['refs']
        lonlats = []
        # picks each element in ref & replaces ID by coordinate tuple (A multiline consist of several points)
        for r in ref:
            # "r" is the ID in Data["Node"], ["lonlat"] a list of [x1,y1] (coordinates)
            lonlat = Data["Node"][str(r)]["lonlat"]
            # try:
            #     print(Data["Node"][str(r)[]])
            lonlat = tuple(lonlat)
            lonlats.append(lonlat)  # a list with tuples
        lonlat_column.append(lonlats)  # adding a new list of tuples every row
    # df_way.drop('refs', axis=1, inplace=True)
    df_way.insert(1, "lonlat", lonlat_column)

In [3]:
def process_line_data(country_code, line_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(country_code, line_data, 'line')
    convert_ways_lines(df_way, Data)
    # Add Type Column
    df_way['Type'] = 'Way'

    # Add Country Column
    df_way['Country'] = AFRICA_CC[country_code]
    return df_way

In [24]:
df_all_substations = pd.DataFrame()
df_all_lines = pd.DataFrame()
df_all_generators = pd.DataFrame()
test_CC = {"NG": "nigeria"}
for country_code in test_CC.keys():
    substation_data, line_data, generator_data = download_and_filter(country_code)
    for feature in ["substation", "line","generator"]:
        if feature == 'substation':
            df_substation = process_substation_data(country_code, substation_data)
            df_all_substations = pd.concat(
                [df_all_substations, df_substation])
        if feature == 'line':
            df_line = process_line_data(country_code, line_data)
            df_all_lines = pd.concat([df_all_lines, df_line])
        if feature == 'generator':
            df_generator = process_generator_data(country_code, generator_data)
            df_all_generators = pd.concat(
                [df_all_generators, df_generator])






Loading Pickle for nigeria


# Substations

In [25]:
df_all_substations

,id,lonlat,tags.power,tags.operator,tags.name,tags.substation,tags.ref,Type,refs,tags.voltage,...,tags.fixme,tags.landuse,tags.source,tags.website,tags.building,tags.addr:city,tags.addr:street,tags.area,tags.access,Country
0,2000011529,"[5.580595199999979, 6.357239900000047]",substation,Transformer,NaN,NaN,NaN,Node,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
1,2161137620,"[5.612709500000012, 6.298418899999993]",substation,Transformer,NaN,NaN,NaN,Node,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
2,2161121796,"[5.605644800000013, 6.292686199999994]",substation,Transformer,NaN,NaN,NaN,Node,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
3,5604720218,"[13.140950299999984, 11.811399999999967]",substation,NaN,NaN,NaN,NaN,Node,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
4,3605463946,"[7.054147699999979, 4.792176100000002]",substation,PHED,Okuru Injection Substation,distribution,NaN,Node,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,553840946,"[7.407928399999965, 6.430845528571455]",substation,NaN,NaN,NaN,NaN,Way,"[5345637324, 5345637323, 5345637322, 534563622...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
164,745794408,"[3.5061223199999914, 6.625164779999999]",substation,NaN,NaN,NaN,NaN,Way,"[6980276932, 6980276933, 6980276934, 698027693...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
165,108020974,"[3.2482386500000224, 6.60293714]",substation,NaN,NaN,transmission,NaN,Way,"[1240204635, 1240204684, 1240204688, 306400478...",330000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria
166,564766375,"[6.32832122857142, 4.990267728571405]",substation,NaN,NaN,NaN,NaN,Way,"[5440610911, 5440610910, 5440610909, 544061090...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nigeria


In [5]:
#----------- SUBSTATIONS -----------

# Clean
df_all_substations.reset_index(drop=True, inplace=True)
df_all_substations.dropna(thresh=len(df_all_substations)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
# df_all_substations.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
# df_all_substations.drop(df_all_substations.loc[df_all_substations['tags.substation']=='industrial'].index, inplace=True)
# df_all_substations.drop(df_all_substations.loc[df_all_substations['tags.substation']=='distribution'].index, inplace=True)

# Generate Files
outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations.')
df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV
gdf_substations = convert_pd_to_gdf(df_all_substations.drop('refs', 1))
gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson

# Generators

In [6]:
# ----------- Generator -----------

df_all_generators.reset_index(drop=True, inplace=True)
df_all_generators.drop(columns = ["tags.fixme","tags.frequency","tags.name:ar","tags.building","tags.barrier"], inplace = True, errors='ignore')
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"}, inplace = True)
df_all_generators.dropna(thresh=len(df_all_generators)*0.25, axis=1, how='all', inplace=True) # Drop Columns with 75% values as N/A

# Generate Files
outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators.')
df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV
gdf_generators = convert_pd_to_gdf(df_all_generators)
gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson

# Lines

In [7]:
# ----------- LINES -----------

# Clean
# TODO: FIX Voltage Filter
# Some transmission lines carry multiple voltages, having voltage_V = 10000;20000  (two lines)
# The following code keeps only the first information before the semicolon..
# Needs to be corrected in future, creating two lines with the same bus ID.


df_all_lines = df_all_lines[
        {
            "id",
            "refs",
            "lonlat",
            "tags.power",
            "tags.cables",
            "tags.voltage",
            "tags.frequency",
            "Type",
            "Country",
        }
    ]

# Clean data    
df_all_lines = df_all_lines.reset_index(drop=True)
df_all_lines = df_all_lines.dropna(subset=['tags.voltage']) # Drop any lines with Voltage = N/A
df_all_lines = df_all_lines.rename(columns = {'tags.voltage':"voltage_V"}) 
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split('*').str[0] #just keeps the 
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split(';').str[0]
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna() ## if cell can't converted to float -> drop
df_all_lines = df_all_lines[df_all_lines.voltage_V > 10000]
# df_all_lines['end_refs'] = 

# Generate Files
outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines.')  
df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV
gdf_lines = convert_pd_to_gdf_lines(df_all_lines.drop('refs', 1), simplified=True)
gdf_lines.to_file(outputfile_partial+'geojson',
            driver="GeoJSON")  # Generate GeoJson

In [8]:
df_line_lookup = df_all_lines[['id','refs']]
display(df_line_lookup)

,id,refs
2,556530547,"[2725719269, 2725912800, 2725719240, 272571923..."
3,671706473,"[6290115060, 6290115059, 6290115058, 629011505..."
6,108020973,"[5198712394, 3064004782, 3064004786, 306400479..."
7,671606207,"[5282666532, 6289294708, 5282666526]"
8,575700599,"[5448403687, 5448403686]"
...,...,...
647,563719558,"[5432136688, 5432136687, 5432136686, 543213668..."
649,565151711,"[5443838099, 5443838095, 5443838091, 544383809..."
650,669957218,"[6274221446, 6274221432, 6274221440, 627422144..."
651,564385545,"[5437840807, 6291806172, 6291806173, 629180617..."


In [9]:
df_node_lookup = df_all_substations[['id','refs']]
df_node_lookup = df_node_lookup.dropna(subset=['refs']) # Drop any nodes with refs = N/A
display(df_node_lookup)

,id,refs
14,564755236,"[5440518752, 5440518751, 5440518750, 544051874..."
15,567535340,"[5462602236, 5462602234, 5462602233, 546260223..."
16,592191538,"[5652509019, 5652509020, 5652509121, 565250912..."
17,564458980,"[5438478937, 5438478936, 5438478935, 543847893..."
18,655273140,"[6139631838, 6139631837, 6139631836, 613963183..."
...,...,...
177,553840946,"[5345637324, 5345637323, 5345637322, 534563622..."
178,745794408,"[6980276932, 6980276933, 6980276934, 698027693..."
179,108020974,"[1240204635, 1240204684, 1240204688, 306400478..."
180,564766375,"[5440610911, 5440610910, 5440610909, 544061090..."


In [10]:
def flatten_df(df):
    out = []
    for n, row in df.iterrows():
        for item in row['refs']:
            row['flat_ref'] = item
            out += [row.copy()]

    flattened_df = pd.DataFrame(out)
    flattened_df.reset_index(drop=True, inplace=True)

    return flattened_df

In [11]:
way_f = flatten_df(df_line_lookup)
display(way_f)

,id,refs,flat_ref
0,556530547,"[2725719269, 2725912800, 2725719240, 272571923...",2725719269
1,556530547,"[2725719269, 2725912800, 2725719240, 272571923...",2725912800
2,556530547,"[2725719269, 2725912800, 2725719240, 272571923...",2725719240
3,556530547,"[2725719269, 2725912800, 2725719240, 272571923...",2725719238
4,556530547,"[2725719269, 2725912800, 2725719240, 272571923...",5368625277
...,...,...,...
32133,695800745,"[6533692645, 6533692646, 6533692647, 653369264...",6533703128
32134,695800745,"[6533692645, 6533692646, 6533692647, 653369264...",6533703129
32135,695800745,"[6533692645, 6533692646, 6533692647, 653369264...",6533703130
32136,695800745,"[6533692645, 6533692646, 6533692647, 653369264...",6533703131


In [12]:
node_f = flatten_df(df_node_lookup)
display(node_f)

,id,refs,flat_ref
0,564755236,"[5440518752, 5440518751, 5440518750, 544051874...",5440518752
1,564755236,"[5440518752, 5440518751, 5440518750, 544051874...",5440518751
2,564755236,"[5440518752, 5440518751, 5440518750, 544051874...",5440518750
3,564755236,"[5440518752, 5440518751, 5440518750, 544051874...",5440518747
4,564755236,"[5440518752, 5440518751, 5440518750, 544051874...",5440518748
...,...,...,...
1231,567184002,"[5460459805, 5460459804, 5460459803, 546045980...",5460459805
1232,567184002,"[5460459805, 5460459804, 5460459803, 546045980...",5460459804
1233,567184002,"[5460459805, 5460459804, 5460459803, 546045980...",5460459803
1234,567184002,"[5460459805, 5460459804, 5460459803, 546045980...",5460459802


In [29]:
df3 = way_f.merge(node_f, on='flat_ref', how='inner')
display(df3)

,id_x,refs_x,flat_ref,id_y,refs_y
0,553772749,"[5345033041, 5432224011, 5345036235, 534503304...",5345033046,553772750,"[5345033057, 5345033048, 5345033049, 534503305..."


In [14]:
all_line_refs = set()
for ref in df_all_lines["refs"]:  # goes through each row in df_way['refs']
    for r in ref:
        all_line_refs.add(r)



In [30]:
all_st_refs = set()
for ref in df_all_substations.dropna(subset=['refs'])["refs"]:  # goes through each row in df_way['refs']
    for r in ref:
        all_st_refs.add(r)

In [20]:
# all_line_refs

In [34]:
all_line_refs.intersection(all_st_refs)

{5345033046}